<a href="https://colab.research.google.com/github/oscar0830/AIAgentClass/blob/dev/Prompt_Engineering_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
TASK = """
Build a Python function parse_ticket(message: str) -> dict that extracts:
- name: customer's name (if present like 'My name is X' or 'I am X')
- order_id: order number patterns like ORD12345 or #12345
- issue_type: one of {shipping, refund, login, hours, other} based on keywords
- escalate: True if message includes angry/urgent keywords (manager, fraud, chargeback, lawsuit)
Rules:
- Always return dict with keys: name, order_id, issue_type, escalate
- Use only Python standard library
"""

# Simulated "code generation attempts" (like an LLM improving over iterations)
CODE_ATTEMPTS = []

# Attempt 1 (buggy / incomplete)
CODE_ATTEMPTS.append(r'''
import re

def parse_ticket(message: str) -> dict:
    # Basic extraction (incomplete)
    name = None
    order_id = None

    m = re.search(r"my name is ([A-Za-z]+)", message.lower())
    if m:
        name = m.group(1)

    o = re.search(r"(ord\d+)", message.lower())
    if o:
        order_id = o.group(1).upper()

    issue_type = "other"
    if "refund" in message.lower():
        issue_type = "refund"
    elif "ship" in message.lower():
        issue_type = "shipping"

    escalate = False
    if "manager" in message.lower():
        escalate = True

    return {"name": name, "order_id": order_id, "issue_type": issue_type, "escalate": escalate}
''')

# Attempt 2 (fixes name patterns, order id patterns, more issue types, escalation keywords)
CODE_ATTEMPTS.append(r'''
import re

def parse_ticket(message: str) -> dict:
    text = message.strip()
    low = text.lower()

    # name extraction: "my name is X" or "i am X"
    name = None
    nm = re.search(r"\b(my name is|i am)\s+([A-Za-z][A-Za-z'-]{1,30})\b", low)
    if nm:
        name = nm.group(2)
        name = name[:1].upper() + name[1:]

    # order_id: ORD12345 or #12345
    order_id = None
    om = re.search(r"\b(ord\d{3,})\b", low)
    if om:
        order_id = om.group(1).upper()
    else:
        hm = re.search(r"#(\d{3,})\b", low)
        if hm:
            order_id = f"#{hm.group(1)}"

    # issue classification
    issue_type = "other"
    if any(k in low for k in ["refund", "return", "money back"]):
        issue_type = "refund"
    elif any(k in low for k in ["ship", "delivery", "tracking", "track"]):
        issue_type = "shipping"
    elif any(k in low for k in ["login", "sign in", "password", "reset"]):
        issue_type = "login"
    elif any(k in low for k in ["hours", "open", "close", "closing time"]):
        issue_type = "hours"

    # escalation signals
    escalate_words = ["manager", "supervisor", "fraud", "chargeback", "lawsuit", "legal"]
    escalate = any(w in low for w in escalate_words)

    return {"name": name, "order_id": order_id, "issue_type": issue_type, "escalate": escalate}
''')

# Attempt 3 (extra robustness: block empty input, avoid false positives, ensure keys always exist)
CODE_ATTEMPTS.append(r'''
import re

def parse_ticket(message: str) -> dict:
    if message is None:
        message = ""
    text = str(message).strip()
    low = text.lower()

    # name extraction
    name = None
    nm = re.search(r"\b(?:my name is|i am)\s+([A-Za-z][A-Za-z'-]{1,30})\b", low)
    if nm:
        raw = nm.group(1)
        name = raw[:1].upper() + raw[1:]

    # order id
    order_id = None
    om = re.search(r"\bORD(\d{3,})\b", text, flags=re.IGNORECASE)
    if om:
        order_id = f"ORD{om.group(1)}"
    else:
        hm = re.search(r"#(\d{3,})\b", text)
        if hm:
            order_id = f"#{hm.group(1)}"

    # issue type
    issue_type = "other"
    rules = [
        ("refund", ["refund", "return", "money back", "exchange"]),
        ("shipping", ["shipping", "ship", "delivery", "tracking", "track", "order status"]),
        ("login", ["login", "log in", "sign in", "password", "reset"]),
        ("hours", ["hours", "open", "close", "closing time"]),
    ]
    for itype, keys in rules:
        if any(k in low for k in keys):
            issue_type = itype
            break

    # escalation
    escalate_words = ["manager", "supervisor", "fraud", "chargeback", "lawsuit", "legal", "scam"]
    escalate = any(w in low for w in escalate_words)

    return {"name": name, "order_id": order_id, "issue_type": issue_type, "escalate": escalate}
''')

In [2]:
import unittest

class TestParseTicket(unittest.TestCase):
    def test_name_my_name_is(self):
        self.assertEqual(parse_ticket("My name is Jordan")["name"], "Jordan")

    def test_name_i_am(self):
        self.assertEqual(parse_ticket("Hi, I am Mia and I need help")["name"], "Mia")

    def test_order_ord(self):
        self.assertEqual(parse_ticket("Where is my order ORD12345?")["order_id"], "ORD12345")

    def test_order_hash(self):
        self.assertEqual(parse_ticket("Order #555 is missing")["order_id"], "#555")

    def test_issue_refund(self):
        self.assertEqual(parse_ticket("I want a refund")["issue_type"], "refund")

    def test_issue_shipping(self):
        self.assertEqual(parse_ticket("Need help with tracking and shipping")["issue_type"], "shipping")

    def test_issue_login(self):
        self.assertEqual(parse_ticket("Can't login, password reset isn't working")["issue_type"], "login")

    def test_issue_hours(self):
        self.assertEqual(parse_ticket("What are your hours? When do you close?")["issue_type"], "hours")

    def test_escalate(self):
        self.assertTrue(parse_ticket("I want to speak to a manager. This is fraud!")["escalate"])

    def test_defaults(self):
        out = parse_ticket("")
        self.assertIn("name", out)
        self.assertIn("order_id", out)
        self.assertIn("issue_type", out)
        self.assertIn("escalate", out)

In [3]:
import io
import traceback

def run_tests():
    stream = io.StringIO()
    runner = unittest.TextTestRunner(stream=stream, verbosity=2)
    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestParseTicket)
    result = runner.run(suite)
    return result.wasSuccessful(), stream.getvalue()

def react_runner(max_iters=3):
    print("=== REACT LOOP START ===")
    print("TASK:", TASK.strip(), "\n")

    for i in range(max_iters):
        print(f"\n--- ITERATION {i+1} ---")

        # PLAN
        print("PLAN:")
        print("- Implement parse_ticket according to the rules")
        print("- Run unit tests")
        print("- If failing, improve parsing rules and re-run\n")

        # GENERATE CODE (simulated)
        code = CODE_ATTEMPTS[i]
        print("GENERATED CODE SNIPPET (first 12 lines):")
        print("\n".join(code.strip().splitlines()[:12]), "\n...")

        # RUN
        env = {}
        try:
            exec(code, env, env)
            # bring parse_ticket into global scope for tests
            globals()["parse_ticket"] = env["parse_ticket"]
        except Exception:
            obs = "CODE EXECUTION ERROR:\n" + traceback.format_exc()
            print(obs)
            continue

        # OBSERVE (tests)
        success, output = run_tests()
        print("\nTEST OUTPUT:\n", output)

        if success:
            print("✅ SUCCESS: All tests passed.")
            print("=== REACT LOOP END ===")
            return code

        print("❌ OBSERVATION: Tests failed. Moving to next iteration (fix).")

    print("\n⚠️ Max iterations reached without passing tests.")
    print("=== REACT LOOP END ===")
    return None

final_code = react_runner(max_iters=3)

=== REACT LOOP START ===
TASK: Build a Python function parse_ticket(message: str) -> dict that extracts:
- name: customer's name (if present like 'My name is X' or 'I am X')
- order_id: order number patterns like ORD12345 or #12345
- issue_type: one of {shipping, refund, login, hours, other} based on keywords
- escalate: True if message includes angry/urgent keywords (manager, fraud, chargeback, lawsuit)
Rules:
- Always return dict with keys: name, order_id, issue_type, escalate
- Use only Python standard library 


--- ITERATION 1 ---
PLAN:
- Implement parse_ticket according to the rules
- Run unit tests
- If failing, improve parsing rules and re-run

GENERATED CODE SNIPPET (first 12 lines):
import re

def parse_ticket(message: str) -> dict:
    # Basic extraction (incomplete)
    name = None
    order_id = None

    m = re.search(r"my name is ([A-Za-z]+)", message.lower())
    if m:
        name = m.group(1)

    o = re.search(r"(ord\d+)", message.lower()) 
...

TEST OUTPUT:
 test_de

In [4]:
print("\n===== FINAL WORKING CODE =====\n")
print(final_code)

print("\n===== DEMO =====")
print(parse_ticket("Hi, my name is Alex. Order ORD7777 hasn't arrived, tracking shows delayed."))
print(parse_ticket("This is fraud. I want a manager. Order #555."))


===== FINAL WORKING CODE =====


import re

def parse_ticket(message: str) -> dict:
    text = message.strip()
    low = text.lower()

    # name extraction: "my name is X" or "i am X"
    name = None
    nm = re.search(r"\b(my name is|i am)\s+([A-Za-z][A-Za-z'-]{1,30})\b", low)
    if nm:
        name = nm.group(2)
        name = name[:1].upper() + name[1:]

    # order_id: ORD12345 or #12345
    order_id = None
    om = re.search(r"\b(ord\d{3,})\b", low)
    if om:
        order_id = om.group(1).upper()
    else:
        hm = re.search(r"#(\d{3,})\b", low)
        if hm:
            order_id = f"#{hm.group(1)}"

    # issue classification
    issue_type = "other"
    if any(k in low for k in ["refund", "return", "money back"]):
        issue_type = "refund"
    elif any(k in low for k in ["ship", "delivery", "tracking", "track"]):
        issue_type = "shipping"
    elif any(k in low for k in ["login", "sign in", "password", "reset"]):
        issue_type = "login"
    elif any(k in 